# ¿Cómo se distribuyen los diferentes tipos de delitos por distrito a lo largo del tiempo en Boston?

# Parte 1

## Introduccion

**Contexto comercial.** Usted es consultor de datos para el Departamento de Policía de Boston. El departamento busca optimizar su estrategia de despliegue policial para que pueda abordar la mayoría de los delitos a medida que ocurren con la menor cantidad de recursos. Además, diferentes grupos dentro del departamento se especializan en diferentes tipos de delitos, por lo que esto debe formar parte de su estrategia general. Les gustaría ver las estadísticas de delincuencia por distrito y por mes, así como visualizarlas en un mapa por tipo de delito y por fecha.

**Problema empresarial.** Su tarea es **crear un tablero interactivo que los jefes de departamento puedan usar para visualizar los delitos de la manera que indicaron anteriormente.** El tablero debe poder usarse a través de un navegador web como Chrome en internet de la empresa.

**Contexto analítico.** En el caso actual, volveremos a utilizar Dash de Plotly para desarrollar el tablero. Esta vez, utilizaremos el conjunto de datos de Kaggle titulado "Crímenes en Boston" que se encuentra aquí: https://www.kaggle.com/AnalyzeBoston/crimes-in-boston. A diferencia del primer caso, juntaremos este caso en un archivo `app.py`. También usaremos una base de datos SQLite como fuente de nuestros datos.

El caso está estructurado de la siguiente manera: tu (1) comprenderás los datos y planificará la configuración adecuada de la aplicación para abordar el problema empresarial; (2) configurar el componente de mapa y varios selectores para filtrar los datos de delitos; (3) configurar algunas parcelas adicionales para visualizar el crimen por geografía y tiempo; y finalmente (4) use nuestra aplicación completa para obtener información.

## Leer data

In [1]:
import sqlite3
import pandas as pd

# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))
# Cerrar conexion
conn.close()
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,None,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,None,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,None,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,None,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,None,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


Tenemos las siguientes variables:

1. **INCIDENT_NUMBER**: The ID of the incident
2. **OFFENSE_CODE**: Unique code for offense
3. **OFFENSE_CODE_GROUP**: Category of offense
4. **OFFENSE_DESCRIPTION**: Longer description of the offense
5. **DISTRICT**: Police district where crime was committed
6. **REPORTING_AREA**: Area where crime was reported
7. **SHOOTING**: Whether guns were fired during the incident
8. **OCCURED_ON_DATE**: Date of incident
9. **YEAR**: Year of incident
10. **MONTH**: Month of incident
11. **DAY_OF_WEEK**: Day of the week of incident
12. **HOUR**: Hour of incident
13. **UCR_PART**: Crime part code
14. **STREET**: Street of incident
15. **Lat**: Location (latitude) of incident
16. **Long**: Location(longitude) of incident
17. **Location**: Tuple of `Lat` and `Long`

### Ejercicio 1

Mirando nuevamente la pregunta y el contexto comercial, ¿cuál de los anteriores sería importante para nosotros?


**Respuesta.** El departamento está interesado en los patrones de delincuencia por distrito y por mes (y no más granular que eso). Esto significa que `DISTRICT` y `MONTH` son importantes para nosotros, pero `DAY_OF_WEEK`, `HOUR`, `REPORTING_AREA` y `STREET` no lo son (porque son demasiado granulares). Sin embargo, `AÑO` es relevante porque es una aclaración de nivel superior de `MES` (por ejemplo, "Diciembre de 2015" es una aclaración importante de "Diciembre"). `OFFENSE_CODE_GROUP` es importante porque el departamento tiene diferentes grupos que se especializan en diferentes tipos de delitos, por lo que analizar eso garantiza que no desplegaremos a los especialistas equivocados en los lugares equivocados.

`Lat` y `Long` son importantes porque para armar un mapa de incidentes, necesitamos la latitud y la longitud. `OCCURRED_ON_DATE` es importante porque el mapa necesita tener un filtro de fecha.

`INCIDENT_NUMBER` y `OFFENSE_CODE` son identificadores y no importan para el análisis de nivel agregado que le interesa al departamento. `OFFENSE_DESCRIPTION` es una versión más granular de `OFFENSE_CODE_GROUP` y probablemente no importe. `SHOOTING` y `UCR_PART` ni siquiera se insinúan como importantes.

## Setting up de app 

Cree un nuevo archivo llamado `app.py`. Comencemos por importar las bibliotecas requeridas por el tablero. Haremos las importaciones principales desde Dash y agregaremos importaciones desde `pandas` y SQLAlchemy para administrar nuestras fuentes de datos. Copie las siguientes líneas en el encabezado de su archivo `app.py`:

In [2]:
import sqlite3
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go

C:\Users\Windows\AppData\Local\Temp\ipykernel_17212\3898493753.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Windows\AppData\Local\Temp\ipykernel_17212\3898493753.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### Ejercicio 2
Escriba el código en su archivo `app.py` que creará una instancia de nuestra aplicación Dash, configurará el token para el diagrama de Mapbox e instanciará el DataFrame que contiene los datos `crime.db` utilizados por los selectores de diseño de nuestra aplicación. (¡Asegúrese de que el archivo `app.py` esté en la misma carpeta que el archivo `crime.db`!)

```
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/uditagarwal/pen/oNvwKNP.css'])

engine = create_engine('sqlite:///crime.db')
df = pd.read_sql("SELECT * from crime", engine.connect(), parse_dates=('OCCURRED_ON_DATE',))
```

In [3]:
import dash
import json
import operator
import random

from functools import reduce

import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.api.types import CategoricalDtype
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))
# Cerrar conexion
conn.close()

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

### Setting up el layout 

El siguiente paso es establecer el objeto de diseño. Vamos a utilizar el mismo CSS y plantilla de diseño que en el caso anterior, que se compone de un encabezado y un cuerpo. El encabezado contiene el título del tablero y el cuerpo contendrá los selectores, gráficos y textos para nuestro tablero.

Comenzaremos configurando dos divs. El primero encerrará el encabezado titulado "Análisis del crimen de Boston", y el otro div representará el cuerpo, que por ahora no tendrá elementos dentro. Copie lo siguiente en su archivo `app.py`:

```
app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Boston Crime Analysis", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[]
    )
])
```

In [4]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))
# Cerrar conexion
conn.close()

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Analisis de crimen en Boston", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[]
    )
])
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 11:38:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:38:23] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:38:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:38:24] "GET /_dash-dependencies HTTP/1.1" 200 -


Nuestra aplicación empieza a tomar forma. Si ejecuta este archivo y navega a la aplicación Dash en su navegador, verá un encabezado y un cuerpo vacío.

### Planning our dashboard (5 mts)

Antes de continuar, debemos decidir cómo dividir en secciones el cuerpo de nuestra aplicación para abordar mejor las necesidades comerciales.

1. Al departamento le gustaría poder visualizar los delitos en un mapa por tipo de delito y por fecha. Por lo tanto, tiene sentido tener algún tipo de filtro desplegable por tipo de delito asociado con el mapa. Y similar al caso anterior, tener un selector de rango de fechas permitiría a los usuarios filtrar por fecha.
2. El departamento también ha declarado que les gustaría ver los delitos por distrito y por mes. Dado que el mes es una variable de serie temporal, tiene sentido mostrar esto en algún tipo de gráfico que muestre de forma natural la progresión del tiempo: me viene a la mente un diagrama de líneas. Dado que el distrito es una variable categórica, tiene sentido utilizar un tipo de gráfico que se categorice fácilmente: un gráfico de barras viene a la mente.

En conjunto, estos implican la siguiente sección de nuestra aplicación:

1. Sección 1
     * Left card - Selector de intervalo de fechas y selector desplegable de tipo de delito
     * Right card - Mapa de Boston
2. Sección 2
     * Left card - Diagrama lineal de incidentes en Boston por mes
     * Right card - Gráfico de barras de incidencias por distrito

## Configurando la primera sección de nuestro tablero

Primero, agreguemos la tarjeta izquierda para la primera sección de nuestra aplicación, que contendrá los selectores. Estos son los filtros para nuestra aplicación y determinan el subconjunto de nuestros datos que se mostrarán en nuestro mapa y parcelas.

Estos componentes se agregarán a un div debajo del cuerpo div agregado anteriormente. Primero agreguemos el selector de intervalo de fechas al diseño:

In [5]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))
# Cerrar conexion
conn.close()

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Analisis de crimen en Boston", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[])
                ]
            )
        ]
    )
])
if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 11:38:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:38:48] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:38:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:38:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:38:49] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -


Tenga en cuenta que agregamos tres divs al cuerpo. La primera se llama `twelve columns card` (porque abarca doce columnas de ancho). Los hijos de este div van a ser dos divs más, `four columns card` y `eight columns card`, respectivamente.

El primer div (`four columns card`) contiene el selector `DatePickerRange` con `start_date` y `end_date` establecidos en los valores mínimo y máximo posibles para una fecha de incidente (`OCCURED_ON_DATE`) de nuestro conjunto de datos.

Reemplace el diseño en su archivo `app.py` con el anterior y ejecútelo para ver el diseño actualizado.

### Ejercicio 3

Agregue un selector `Dropdown` al div `four columns card` con id `study-dropdown` justo encima del selector de intervalo de fechas. El selector debe contener todos los códigos de infracción únicos enumerados en la columna `OFFENSE_CODE_GROUP` en nuestro conjunto de datos. Establezca el valor predeterminado de este delito en "Larceny (Hurto)". Use los documentos de Dash como referencia en https://dash.plot.ly/dash-core-components/dropdown.

```
app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Boston Crime Analysis", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[])
                ]
            )
        ]
    )
])
```

In [6]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))
# Cerrar conexion
conn.close()

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Boston Crime Analysis", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[])
                ]
            )
        ]
    )
])

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 11:39:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:39:17] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:39:17] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:39:17] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:39:17] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:39:17] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -


## Agregar el mapa de crimenes

Agreguemos el mapa a nuestro tablero usando el diagrama de dispersión de Mapbox (https://plot.ly/python/scattermapbox/). Agregaremos el diagrama del mapa al div`eight columns card`.

**NOTA: Nuestro conjunto de datos tiene más de 300 000 filas. Si cargamos todos estos puntos de datos a la vez, explotará el mapa. Para solucionar este problema, solo trazaremos los puntos de datos correspondientes a los grupos delictivos seleccionados en nuestro tablero en el filtro de selección de delitos.**

```
app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Boston Crime Analysis", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[
                        html.H1(children="Geographical Map of Crimes in Boston", style={'textAlign': 'center'}),
                            dcc.Graph(
                                id='map-plot',
                                figure={ 
                                    'data': [go.Scattermapbox()],
                                    'layout': go.Layout(
                                            mapbox_style="dark",
                                            mapbox_accesstoken=token,
                                            mapbox_zoom=10,
                                            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                                            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
                                        )
                                }
                        )
                    ])
                ]
            )
        ]
    )
])
```

In [7]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))
# Cerrar conexion
conn.close()

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Analisis de crimenes en Boston", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[
                        html.H1(children="Mapa geografico de crimenes en Boston", style={'textAlign': 'center'}),
                            dcc.Graph(
                                id='map-plot',
                                figure={ 
                                    'data': [go.Scattermapbox()],
                                    'layout': go.Layout(
                                            mapbox_style="dark",
                                            mapbox_accesstoken=token,
                                            mapbox_zoom=10,
                                            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                                            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
                                        )
                                }
                        )
                    ])
                ]
            )
        ]
    )
])

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 11:39:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:39:41] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:39:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:39:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:39:41] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:39:41] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:39:41] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:39:41] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


Actualmente, el mapa no tiene ningún dato pasado para su visualización. Estaremos actualizando los puntos que se muestran en el mapa usando una función callback que toma entradas de los selectores `Dropdown` y `DatePickerRange`. Agreguemos el esqueleto de la función callback a nuestra aplicación:

In [ ]:
# NOTE:No correr solo para explicar
@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_map(start_date, end_date, value):
    return { 
        'data': locations_by_crimetype(value, start_date, end_date),
        'layout': go.Layout(
            mapbox_style="dark",
            mapbox_accesstoken=token,
            mapbox_zoom=10,
            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
        )
    }

Tomemos un poco para hablar sobre el código anterior. Creamos una función llamada `update_crimes_map()` con el decorador `app.callback()`. Las entradas al decorador son:

1. entrada de rango de fechas como valor `start_date`
2. entrada de rango de fechas como valor de `fecha de finalización`
3. menú desplegable de tipo de delito con `valor`

La salida de la función actualiza el `map-plot`. La función de devolución de llamada deberá devolver el valor de `figure` en el gráfico del mapa. El valor de `figure` debe ser los datos (que es el gráfico) y el diseño de este gráfico.

Como habrás notado, el atributo `data` llama a la función `locations_by_crimetype()` que aún no existe.

Configuremos esta función que devolverá un objeto `go.Scattermapbox()` que contiene información sobre la latitud y longitud de los incidentes. Para hacer esto, ejecutaremos una consulta SQL para filtrar las filas de nuestra fuente de datos según el selector de rango de datos y el selector desplegable de tipo de delito. Tenga en cuenta que nuestro selector de tipo de delito de valores múltiples puede pasar más de un tipo de delito al mapa.

### Ejercicio 4

Escriba la función `get_filtered_rows()`, que toma `crime_type`, `start_date` y `end_date` como entradas y devuelve un DataFrame `pandas` que contiene las filas de nuestra fuente de datos que se encuentran dentro de ese `crime_type` y entre `start_date ` y `end_date`. (Sugerencia: es posible que deba buscar la sintaxis de la función `read_sql()` para esto).

In [8]:
def get_filtered_row(crime_type, start_date, end_date):
    filtered_df = df[(df['OCCURRED_ON_DATE'] >= start_date) & (df['OCCURRED_ON_DATE'] <= end_date)]
    if isinstance(crime_type, list):
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'].isin(crime_type)]
    else:
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'] == crime_type]
    return filtered_df

In [10]:
# Ejemplo
get_filtered_row(crime_type=['Larceny','Vandalism'], start_date=df.OCCURRED_ON_DATE.min(), end_date=df.OCCURRED_ON_DATE.max()).head()
get_filtered_row(crime_type='Larceny', start_date=df.OCCURRED_ON_DATE.min(), end_date=df.OCCURRED_ON_DATE.max()).head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,None,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
36,I182070892,613,Larceny,LARCENY SHOPLIFTING,B2,276,None,2018-09-03 17:54:00,2018,9,Monday,17,Part One,SHAWMUT AVE,42.333183,-71.083076,"(42.33318299, -71.08307607)"
45,I182070885,619,Larceny,LARCENY ALL OTHERS,B3,456,None,2018-09-03 17:17:00,2018,9,Monday,17,Part One,ELLINGTON ST,42.301546,-71.081182,"(42.30154555, -71.08118184)"
54,I182070874,617,Larceny,LARCENY THEFT FROM BUILDING,A1,102,None,2018-09-03 16:00:00,2018,9,Monday,16,Part One,WINTER ST,42.356024,-71.061776,"(42.35602373, -71.06177615)"
60,I182070868,616,Larceny,LARCENY THEFT OF BICYCLE,D14,790,None,2018-09-03 06:15:00,2018,9,Monday,6,Part One,CHESTNUT HILL AVE,42.335400,-71.150257,"(42.33539970, -71.15025675)"


Ahora podemos usar esta función para escribir la función `locations_by_crimetype()`:

In [11]:
import random

def locations_by_crimetype(crime_type, start_date, end_date):
    data = [] # Output of the function is an array
    
    df = get_filtered_row(crime_type, start_date, end_date)
    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        color = "%06x" % random.randint(0, 0xFFFFFF)
        data.append(
            go.Scattermapbox(
                lat=group['Lat'],
                lon=group['Long'],
                mode='markers',
                marker={
                    'color': '#' + color,
                },
                text=group['OFFENSE_DESCRIPTION'],
                name=name
            )
        )
    return data

In [12]:
locations_by_crimetype(crime_type=['Larceny','Vandalism'], start_date=df.OCCURRED_ON_DATE.min(), end_date=df.OCCURRED_ON_DATE.max())

[Scattermapbox({
     'lat': array([42.35779134, 42.33318299, 42.30154555, ..., 42.33555954, 42.30703835,
                   42.33695098]),
     'lon': array([-71.13937053, -71.08307607, -71.08118184, ..., -71.07436364,
                   -71.06615319, -71.08574813]),
     'marker': {'color': '#915de8'},
     'mode': 'markers',
     'name': 'Larceny',
     'text': array(['LARCENY ALL OTHERS', 'LARCENY SHOPLIFTING', 'LARCENY ALL OTHERS', ...,
                    'LARCENY SHOPLIFTING $50 TO $199', 'LARCENY OTHER $200 & OVER',
                    'LARCENY OTHER $50 TO $199'], dtype=object)
 }),
 Scattermapbox({
     'lat': array([42.30682138, 42.34385799, 42.29848866, ..., 42.26255067, 42.28526024,
                   42.28526024]),
     'lon': array([-71.06030035, -71.07953502, -71.06313294, ..., -71.11582205,
                   -71.06444063, -71.06444063]),
     'marker': {'color': '#430ab6'},
     'mode': 'markers',
     'name': 'Vandalism',
     'text': array(['VANDALISM', 'VANDALISM',

In [16]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

def get_filtered_row(crime_type, start_date, end_date):
    filtered_df = df[(df['OCCURRED_ON_DATE'] >= start_date) & (df['OCCURRED_ON_DATE'] <= end_date)]
    if isinstance(crime_type, list):
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'].isin(crime_type)]
    else:
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'] == crime_type]
    return filtered_df

import random

def locations_by_crimetype(crime_type, start_date, end_date):
    data = [] # Output of the function is an array
    
    df = get_filtered_row(crime_type, start_date, end_date)
    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        color = "%06x" % random.randint(0, 0xFFFFFF)
        data.append(
            go.Scattermapbox(
                lat=group['Lat'],
                lon=group['Long'],
                mode='markers',
                marker={
                    'color': '#' + color,
                },
                text=group['OFFENSE_DESCRIPTION'],
                name=name
            )
        )
    return data

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Analisis de crimenes en Boston", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[
                        html.H1(children="Mapa geografico de crimenes en Boston", style={'textAlign': 'center'}),
                            dcc.Graph(
                                id='map-plot',
                                figure={ 
                                    'data': [go.Scattermapbox()],
                                    'layout': go.Layout(
                                            mapbox_style="dark",
                                            mapbox_accesstoken=token,
                                            mapbox_zoom=10,
                                            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                                            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
                                        )
                                }
                        )
                    ])
                ]
            )
        ]
    )
])

@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_map(start_date, end_date, value):
    print(locations_by_crimetype(value, start_date, end_date))
    return { 
            'data': locations_by_crimetype(value, start_date, end_date),
            'layout': go.Layout(
                mapbox_style="dark",
                mapbox_accesstoken=token,
                mapbox_zoom=10,
                margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
            )
        }


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET /_favicon.ico?v=2.9.3 HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:51:59] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


[Scattermapbox({
    'lat': array([42.35779134, 42.33318299, 42.30154555, ..., 42.33555954, 42.30703835,
                  42.33695098]),
    'lon': array([-71.13937053, -71.08307607, -71.08118184, ..., -71.07436364,
                  -71.06615319, -71.08574813]),
    'marker': {'color': '#11c094'},
    'mode': 'markers',
    'name': 'Larceny',
    'text': array(['LARCENY ALL OTHERS', 'LARCENY SHOPLIFTING', 'LARCENY ALL OTHERS', ...,
                   'LARCENY SHOPLIFTING $50 TO $199', 'LARCENY OTHER $200 & OVER',
                   'LARCENY OTHER $50 TO $199'], dtype=object)
})]


127.0.0.1 - - [23/Apr/2023 11:52:00] "POST /_dash-update-component HTTP/1.1" 200 -


[Scattermapbox({
    'lat': array([42.35779134, 42.33318299, 42.30154555, ..., 42.33555954, 42.30703835,
                  42.33695098]),
    'lon': array([-71.13937053, -71.08307607, -71.08118184, ..., -71.07436364,
                  -71.06615319, -71.08574813]),
    'marker': {'color': '#64fc44'},
    'mode': 'markers',
    'name': 'Larceny',
    'text': array(['LARCENY ALL OTHERS', 'LARCENY SHOPLIFTING', 'LARCENY ALL OTHERS', ...,
                   'LARCENY SHOPLIFTING $50 TO $199', 'LARCENY OTHER $200 & OVER',
                   'LARCENY OTHER $50 TO $199'], dtype=object)
}), Scattermapbox({
    'lat': array([42.30682138, 42.34385799, 42.29848866, ..., 42.26255067, 42.28526024,
                  42.28526024]),
    'lon': array([-71.06030035, -71.07953502, -71.06313294, ..., -71.11582205,
                  -71.06444063, -71.06444063]),
    'marker': {'color': '#9a8d2c'},
    'mode': 'markers',
    'name': 'Vandalism',
    'text': array(['VANDALISM', 'VANDALISM', 'VANDALISM', ..., '

127.0.0.1 - - [23/Apr/2023 11:53:44] "POST /_dash-update-component HTTP/1.1" 200 -


### Pregunta:

Usando la última versión de su archivo `app.py`, vea si puede encontrar la región donde:

1. ¿Ocurrieron la mayoría de las infracciones de licencia?
2. ¿Ocurrieron la mayoría de las infracciones aéreas?
3. ¿Ocurrió la mayor cantidad de violaciones de licor?


# Parte 2

## Configuración de la sección 2 de nuestro tablero 

Ahora podemos proceder a configurar la siguiente sección de nuestro tablero, que proporcionará estadísticas de delincuencia por distrito y por mes. Agregaremos otro div llamado `twelve columns card 2` que contiene dos divs de seis columnas de ancho:

1. El primer div será un gráfico de líneas que realiza un seguimiento del número total de incidentes por mes a lo largo del tiempo.
2. El segundo div será un gráfico de barras del número de incidentes por distrito

Ambos se filtrarán en función de la elección del selector desplegable del tipo de delito.

### Ejercicio 5


Agregue las siguientes dos gráficas a los divs `six columns card` y`six columns card 2`:

1. Primer div - `go.Scatter()` con el id `crime-total-graph` (recuerde, `go.Scatter()` se usa para generar un diagrama de líneas en Dash!)
2. Segundo div - `go.Bar()` con el id `crime-district-graph`

Por ahora, no se preocupe por pasar datos a estos gráficos. Más tarde, escribiremos funciones callbacksen nuestros selectores de entrada para actualizarlas.

In [17]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

def get_filtered_row(crime_type, start_date, end_date):
    filtered_df = df[(df['OCCURRED_ON_DATE'] >= start_date) & (df['OCCURRED_ON_DATE'] <= end_date)]
    if isinstance(crime_type, list):
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'].isin(crime_type)]
    else:
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'] == crime_type]
    return filtered_df

import random

def locations_by_crimetype(crime_type, start_date, end_date):
    data = [] # Output of the function is an array
    
    df = get_filtered_row(crime_type, start_date, end_date)
    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        color = "%06x" % random.randint(0, 0xFFFFFF)
        data.append(
            go.Scattermapbox(
                lat=group['Lat'],
                lon=group['Long'],
                mode='markers',
                marker={
                    'color': '#' + color,
                },
                text=group['OFFENSE_DESCRIPTION'],
                name=name
            )
        )
    return data

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Boston Crime Analysis", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[
                        html.H1(children="Geographical Map of Crimes in Boston", style={'textAlign': 'center'}),
                            dcc.Graph(
                                id='map-plot',
                                figure={ 
                                    'data': [go.Scattermapbox()],
                                    'layout': go.Layout(
                                            mapbox_style="dark",
                                            mapbox_accesstoken=token,
                                            mapbox_zoom=10,
                                            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                                            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
                                        )
                                }
                        )
                    ])
                ]
            ),
            html.Div(
                className='twelve columns card 2',
                children=[
                    html.Div(
                        className='six columns card',
                        children=[
                            dcc.Graph(
                            id='crime-total-graph',
                            figure={
                                'data': [go.Scatter()],
                            }
                            )
                        ]
                    ),
                    html.Div(
                        className='six columns card 2',
                        children=[
                            dcc.Graph(
                                id='crime-district-graph',
                                figure={
                                    'data': [go.Bar()],
                                }
                            )
                        ]
                    )
                ]
            )
        ]
    )
])

@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_map(start_date, end_date, value):
    return { 
            'data': locations_by_crimetype(value, start_date, end_date),
            'layout': go.Layout(
                mapbox_style="dark",
                mapbox_accesstoken=token,
                mapbox_zoom=10,
                margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
            )
        }


if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 11:55:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:55:49] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:55:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:55:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:55:49] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:55:49] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:55:49] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:55:49] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:55:50] "POST /_dash-update-component HTTP/1.1" 200 -


## Configuración de la función callback para el gráfico de delitos por mes 

Para actualizar los datos en nuestro diagrama de líneas con id `crime-total-graph` (recuerde, un diagrama de líneas en Dash se crea usando `go.Scatter()`), crearemos una nueva función de devolución de llamada que toma la configuración de nuestros selectores como entrada

### Ejercicio 6

Escriba el esqueleto de dicha función de devolución de llamada, que llamaremos `update_crimes_total()`. Suponga que ya existe otra función `crimes_by_year()` que puede usar y que devuelve los datos necesarios para su gráfico de líneas.

```
@app.callback(
    dash.dependencies.Output('crime-total-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_line_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_year(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Crime Occurence over Time'
            }
        }
    }
```

Por supuesto, esto no funcionará porque aún no se ha definido `crimes_by_year()`. Vamos a desarrollarlo para que pueda devolver correctamente los datos que necesita nuestro diagrama de líneas:

In [18]:
def crimes_by_year(crime_type, start_date, end_date):
    data = []
    df = get_filtered_row(crime_type, start_date, end_date)

    df['YearMonth'] = pd.to_datetime(df['OCCURRED_ON_DATE'].map(lambda x: "{}-{}".format(x.year, x.month)))

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('YearMonth', as_index=False).count()
        data.append(
            go.Scatter(x=grouped['YearMonth'], y=grouped['Lat'], name=name)
        )
    return data

In [20]:
crimes_by_year(crime_type='Vandalism',start_date=df.OCCURRED_ON_DATE.min(), end_date=df.OCCURRED_ON_DATE.max())

[Scatter({
     'name': 'Vandalism',
     'x': array([datetime.datetime(2015, 6, 1, 0, 0),
                 datetime.datetime(2015, 7, 1, 0, 0),
                 datetime.datetime(2015, 8, 1, 0, 0),
                 datetime.datetime(2015, 9, 1, 0, 0),
                 datetime.datetime(2015, 10, 1, 0, 0),
                 datetime.datetime(2015, 11, 1, 0, 0),
                 datetime.datetime(2015, 12, 1, 0, 0),
                 datetime.datetime(2016, 1, 1, 0, 0),
                 datetime.datetime(2016, 2, 1, 0, 0),
                 datetime.datetime(2016, 3, 1, 0, 0),
                 datetime.datetime(2016, 4, 1, 0, 0),
                 datetime.datetime(2016, 5, 1, 0, 0),
                 datetime.datetime(2016, 6, 1, 0, 0),
                 datetime.datetime(2016, 7, 1, 0, 0),
                 datetime.datetime(2016, 8, 1, 0, 0),
                 datetime.datetime(2016, 9, 1, 0, 0),
                 datetime.datetime(2016, 10, 1, 0, 0),
                 datetime.datetime(2016, 

In [21]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

def get_filtered_row(crime_type, start_date, end_date):
    filtered_df = df[(df['OCCURRED_ON_DATE'] >= start_date) & (df['OCCURRED_ON_DATE'] <= end_date)]
    if isinstance(crime_type, list):
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'].isin(crime_type)]
    else:
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'] == crime_type]
    return filtered_df

import random

def locations_by_crimetype(crime_type, start_date, end_date):
    data = [] # Output of the function is an array
    
    df = get_filtered_row(crime_type, start_date, end_date)
    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        color = "%06x" % random.randint(0, 0xFFFFFF)
        data.append(
            go.Scattermapbox(
                lat=group['Lat'],
                lon=group['Long'],
                mode='markers',
                marker={
                    'color': '#' + color,
                },
                text=group['OFFENSE_DESCRIPTION'],
                name=name
            )
        )
    return data


def crimes_by_year(crime_type, start_date, end_date):
    data = []
    df = get_filtered_row(crime_type, start_date, end_date)

    df['YearMonth'] = pd.to_datetime(df['OCCURRED_ON_DATE'].map(lambda x: "{}-{}".format(x.year, x.month)))

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('YearMonth', as_index=False).count()
        data.append(
            go.Scatter(x=grouped['YearMonth'], y=grouped['Lat'], name=name)
        )
    return data


app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Boston Crime Analysis", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[
                        html.H1(children="Geographical Map of Crimes in Boston", style={'textAlign': 'center'}),
                            dcc.Graph(
                                id='map-plot',
                                figure={ 
                                    'data': [go.Scattermapbox()],
                                    'layout': go.Layout(
                                            mapbox_style="dark",
                                            mapbox_accesstoken=token,
                                            mapbox_zoom=10,
                                            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                                            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
                                        )
                                }
                        )
                    ])
                ]
            ),
            html.Div(
                className='twelve columns card 2',
                children=[
                    html.Div(
                        className='six columns card',
                        children=[
                            dcc.Graph(
                            id='crime-total-graph',
                            figure={
                                'data': [go.Scatter()],
                            }
                            )
                        ]
                    ),
                    html.Div(
                        className='six columns card 2',
                        children=[
                            dcc.Graph(
                                id='crime-district-graph',
                                figure={
                                    'data': [go.Bar()],
                                }
                            )
                        ]
                    )
                ]
            )
        ]
    )
])

@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_map(start_date, end_date, value):
    return { 
            'data': locations_by_crimetype(value, start_date, end_date),
            'layout': go.Layout(
                mapbox_style="dark",
                mapbox_accesstoken=token,
                mapbox_zoom=10,
                margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
            )
        }

@app.callback(
    dash.dependencies.Output('crime-total-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_line_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_year(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Ocurrencia de crimen en el tiempo'
            }
        }
    }

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 11:58:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:58:12] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:58:12] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:58:12] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:58:13] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:58:13] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:58:13] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:58:13] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:58:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:58:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:58:22] "PO

## Configuración de la función callbackpara el gráfico de barras

Continuemos configurando también la funcion callback para el gráfico de barras. Ahora que lo ha visto hacer una vez para el gráfico de líneas, le dejaremos esto a usted.

### Ejercicio 7

Cree una función callback para el gráfico de barras con id `crime-district-graph`. Esto debe mostrar la cantidad de incidentes por distrito, filtrados por los valores en el selector de tipo de delito de la sección 1.

Queremos que sea un gráfico de barras horizontales, con el eje x representando el número de incidentes y el eje y representando los nombres de los distritos. Consulte https://plot.ly/python/bar-charts/ para conocer los parámetros de un gráfico de barras.

```
def crimes_by_district(crime_type, start_date, end_date):
    data = []
    df = get_filtered_rows(crime_type, start_date, end_date)

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('DISTRICT', as_index=False).count()
        data.append(
            go.Bar(y=grouped['DISTRICT'], x=grouped['Lat'].sort_values(), name=name, orientation='h')
        )
    return data


@app.callback(
    dash.dependencies.Output('crime-district-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_bar_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_district(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Ocurrencia de Crimen por distrito',

            }
        }
    }
```

In [22]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

def get_filtered_row(crime_type, start_date, end_date):
    filtered_df = df[(df['OCCURRED_ON_DATE'] >= start_date) & (df['OCCURRED_ON_DATE'] <= end_date)]
    if isinstance(crime_type, list):
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'].isin(crime_type)]
    else:
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'] == crime_type]
    return filtered_df

import random

def locations_by_crimetype(crime_type, start_date, end_date):
    data = [] # Output of the function is an array
    
    df = get_filtered_row(crime_type, start_date, end_date)
    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        color = "%06x" % random.randint(0, 0xFFFFFF)
        data.append(
            go.Scattermapbox(
                lat=group['Lat'],
                lon=group['Long'],
                mode='markers',
                marker={
                    'color': '#' + color,
                },
                text=group['OFFENSE_DESCRIPTION'],
                name=name
            )
        )
    return data


def crimes_by_year(crime_type, start_date, end_date):
    data = []
    df = get_filtered_row(crime_type, start_date, end_date)

    df['YearMonth'] = pd.to_datetime(df['OCCURRED_ON_DATE'].map(lambda x: "{}-{}".format(x.year, x.month)))

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('YearMonth', as_index=False).count()
        data.append(
            go.Scatter(x=grouped['YearMonth'], y=grouped['Lat'], name=name)
        )
    return data

def crimes_by_district(crime_type, start_date, end_date):
    data = []
    df = get_filtered_row(crime_type, start_date, end_date)

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('DISTRICT', as_index=False).count()
        data.append(
            go.Bar(y=grouped['DISTRICT'], x=grouped['Lat'].sort_values(), name=name, orientation='h')
        )
    return data

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Boston Crime Analysis", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[
                        html.H1(children="Geographical Map of Crimes in Boston", style={'textAlign': 'center'}),
                            dcc.Graph(
                                id='map-plot',
                                figure={ 
                                    'data': [go.Scattermapbox()],
                                    'layout': go.Layout(
                                            mapbox_style="dark",
                                            mapbox_accesstoken=token,
                                            mapbox_zoom=10,
                                            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                                            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
                                        )
                                }
                        )
                    ])
                ]
            ),
            html.Div(
                className='twelve columns card 2',
                children=[
                    html.Div(
                        className='six columns card',
                        children=[
                            dcc.Graph(
                            id='crime-total-graph',
                            figure={
                                'data': [go.Scatter()],
                            }
                            )
                        ]
                    ),
                    html.Div(
                        className='six columns card 2',
                        children=[
                            dcc.Graph(
                                id='crime-district-graph',
                                figure={
                                    'data': [go.Bar()],
                                }
                            )
                        ]
                    )
                ]
            )
        ]
    )
])

@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_map(start_date, end_date, value):
    return { 
            'data': locations_by_crimetype(value, start_date, end_date),
            'layout': go.Layout(
                mapbox_style="dark",
                mapbox_accesstoken=token,
                mapbox_zoom=10,
                margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
            )
        }

@app.callback(
    dash.dependencies.Output('crime-total-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_line_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_year(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Ocurrencia de crimen en el tiempo'
            }
        }
    }

@app.callback(
    dash.dependencies.Output('crime-district-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_bar_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_district(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Ocurrencia de Crimen por distrito',

            }
        }
    }

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 11:59:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:59:47] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:59:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:59:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:59:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:59:47] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:59:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:59:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 11:59:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:59:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 11:59:49] "PO

### Pregunta:

Usando la última versión de su archivo `app.py`, vea si puede responder las siguientes preguntas:

1. ¿En qué meses hay la menor cantidad de delitos? ¿Este patrón es consistente en varios tipos de delitos?
2. ¿Cuál es la tendencia a lo largo del tiempo en los delitos de "robo residencial (Residential Burglaries)"?
3. ¿Qué distrito tiene el mayor número de incidentes de Hurto (Larceny)?


## Agregando datos horarios para cada día de la semana

Presenta su aplicación a los jefes de departamento, quienes están muy satisfechos con los resultados. Sin embargo, les gustaría que agregara una cosa más: quieren ver, para cada día de la semana, un diagrama de líneas que represente el número total de incidentes por hora durante ese día de la semana durante el período de tiempo seleccionado. Es decir, quieren ver *subparcelas* (https://plot.ly/python/subplots/) para cada día de la semana.

Agreguemos un nuevo div a nuestro diseño con una trama vacía llamada `crimes-weekly`:

In [23]:
# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

def get_filtered_row(crime_type, start_date, end_date):
    filtered_df = df[(df['OCCURRED_ON_DATE'] >= start_date) & (df['OCCURRED_ON_DATE'] <= end_date)]
    if isinstance(crime_type, list):
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'].isin(crime_type)]
    else:
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'] == crime_type]
    return filtered_df

import random

def locations_by_crimetype(crime_type, start_date, end_date):
    data = [] # Output of the function is an array
    
    df = get_filtered_row(crime_type, start_date, end_date)
    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        color = "%06x" % random.randint(0, 0xFFFFFF)
        data.append(
            go.Scattermapbox(
                lat=group['Lat'],
                lon=group['Long'],
                mode='markers',
                marker={
                    'color': '#' + color,
                },
                text=group['OFFENSE_DESCRIPTION'],
                name=name
            )
        )
    return data


def crimes_by_year(crime_type, start_date, end_date):
    data = []
    df = get_filtered_row(crime_type, start_date, end_date)

    df['YearMonth'] = pd.to_datetime(df['OCCURRED_ON_DATE'].map(lambda x: "{}-{}".format(x.year, x.month)))

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('YearMonth', as_index=False).count()
        data.append(
            go.Scatter(x=grouped['YearMonth'], y=grouped['Lat'], name=name)
        )
    return data

def crimes_by_district(crime_type, start_date, end_date):
    data = []
    df = get_filtered_row(crime_type, start_date, end_date)

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('DISTRICT', as_index=False).count()
        data.append(
            go.Bar(y=grouped['DISTRICT'], x=grouped['Lat'].sort_values(), name=name, orientation='h')
        )
    return data

app.layout = html.Div(children=[
    html.Div(
        children=[
            html.H2(children="Boston Crime Analysis", className='h2-title'),
        ],
        className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(className="eight columns card", children=[
                        html.H1(children="Geographical Map of Crimes in Boston", style={'textAlign': 'center'}),
                            dcc.Graph(
                                id='map-plot',
                                figure={ 
                                    'data': [go.Scattermapbox()],
                                    'layout': go.Layout(
                                            mapbox_style="dark",
                                            mapbox_accesstoken=token,
                                            mapbox_zoom=10,
                                            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                                            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
                                        )
                                }
                        )
                    ])
                ]
            ),
            html.Div(
                className='twelve columns card 2',
                children=[
                    html.Div(
                        className='six columns card',
                        children=[
                            dcc.Graph(
                            id='crime-total-graph',
                            figure={
                                'data': [go.Scatter()],
                            }
                            )
                        ]
                    ),
                    html.Div(
                        className='six columns card 2',
                        children=[
                            dcc.Graph(
                                id='crime-district-graph',
                                figure={
                                    'data': [go.Bar()],
                                }
                            )
                        ]
                    )
                ]
            ),
            html.Div(
                className='twelve columns card 3',
                children=[
                    dcc.Graph(
                        id="crimes-weekly",
                        figure={
                            'data': [go.Scatter()]
                        }
                    )
                ]
            )
        ]
    )
])

@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_map(start_date, end_date, value):
    return { 
            'data': locations_by_crimetype(value, start_date, end_date),
            'layout': go.Layout(
                mapbox_style="dark",
                mapbox_accesstoken=token,
                mapbox_zoom=10,
                margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
            )
        }

@app.callback(
    dash.dependencies.Output('crime-total-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_line_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_year(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Ocurrencia de crimen en el tiempo'
            }
        }
    }

@app.callback(
    dash.dependencies.Output('crime-district-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_bar_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_district(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Ocurrencia de Crimen por distrito',

            }
        }
    }

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 12:00:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:00:43] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:00:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:00:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:00:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:00:43] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:00:43] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:00:43] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:00:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:00:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:00:46] "PO

### Agregar la función callback para nuestros diagramas de líneas

Configuraremos una función de devolución de llamada simple que actualiza el gráfico `crimes-weekly`:

Nuevamente, subcontratamos el trabajo pesado de esta función a una función diferente `crimes_week()`:

In [6]:
# NOTE: Do NOT run this cell!!! It is for instructional purposes only - it will NOT work!
@app.callback(
    dash.dependencies.Output('crimes-weekly', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_scatter_plot(start_date, end_date, value):
    return crimes_week(value, start_date, end_date)
from plotly.subplots import make_subplots
from pandas.api.types import CategoricalDtype

def crimes_week(crime_type, start_date, end_date):
    dff = get_filtered_rows(crime_type, start_date, end_date)
    days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    fig = make_subplots(rows=1, cols=7, subplot_titles=days)
    
    
    cat_type = CategoricalDtype(categories=days, ordered=True)
    dff['DAY_OF_WEEK'] = dff['DAY_OF_WEEK'].astype(cat_type)

    for crime_name, group in dff.groupby('OFFENSE_CODE_GROUP'):
        i=1
        for day_of_week, week_group in group.groupby('DAY_OF_WEEK'):
            hour_group = week_group.groupby('HOUR', as_index=False).count()
            chart = go.Scatter(x=hour_group['HOUR'], y=hour_group['Lat'], name=crime_name)
            fig.append_trace(chart, row=1, col=i)
            i += 1
    
    fig.update_layout(
        title="Hourly/Weekly Crime Trends",
    )
    return fig

Ahora si nuestra version final de la app

In [24]:
import dash
import json
import operator
import random

from functools import reduce

import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from sqlalchemy import create_engine
from plotly.subplots import make_subplots
from pandas.api.types import CategoricalDtype


# Connect to SQL Engine 
import sqlite3
# Conexion a SQLite database
conn = sqlite3.connect('crime.db')
# Guardar resultados de query en DF
df = pd.read_sql_query('SELECT * FROM crime', conn,parse_dates=('OCCURRED_ON_DATE',))

# Obtener Token para Mapbox & Leer GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)

def get_filtered_row(crime_type, start_date, end_date):
    filtered_df = df[(df['OCCURRED_ON_DATE'] >= start_date) & (df['OCCURRED_ON_DATE'] <= end_date)]
    if isinstance(crime_type, list):
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'].isin(crime_type)]
    else:
        filtered_df = filtered_df[filtered_df['OFFENSE_CODE_GROUP'] == crime_type]
    return filtered_df

import random

def locations_by_crimetype(crime_type, start_date, end_date):
    data = [] # Output of the function is an array
    
    df = get_filtered_row(crime_type, start_date, end_date)
    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        color = "%06x" % random.randint(0, 0xFFFFFF)
        data.append(
            go.Scattermapbox(
                lat=group['Lat'],
                lon=group['Long'],
                mode='markers',
                marker={
                    'color': '#' + color,
                },
                text=group['OFFENSE_DESCRIPTION'],
                name=name
            )
        )
    return data


def crimes_by_year(crime_type, start_date, end_date):
    data = []
    df = get_filtered_row(crime_type, start_date, end_date)

    df['YearMonth'] = pd.to_datetime(df['OCCURRED_ON_DATE'].map(lambda x: "{}-{}".format(x.year, x.month)))

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('YearMonth', as_index=False).count()
        data.append(
            go.Scatter(x=grouped['YearMonth'], y=grouped['Lat'], name=name)
        )
    return data

def crimes_by_district(crime_type, start_date, end_date):
    data = []
    df = get_filtered_row(crime_type, start_date, end_date)

    for name, group in df.groupby('OFFENSE_CODE_GROUP'):
        grouped = group.groupby('DISTRICT', as_index=False).count()
        data.append(
            go.Bar(y=grouped['DISTRICT'], x=grouped['Lat'].sort_values(), name=name, orientation='h')
        )
    return data


def crimes_week(crime_type, start_date, end_date):
    cats = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    fig = make_subplots(rows=1, cols=7, subplot_titles=cats)
    dff = get_filtered_row(crime_type, start_date, end_date)
    
    cat_type = CategoricalDtype(categories=cats, ordered=True)
    dff['DAY_OF_WEEK'] = dff['DAY_OF_WEEK'].astype(cat_type)

    for crime_name, group in dff.groupby('OFFENSE_CODE_GROUP'):
        i=1
        for day_of_week, week_group in group.groupby('DAY_OF_WEEK'):
            hour_group = week_group.groupby('HOUR', as_index=False).count()
            fig.append_trace(
                go.Scatter(x=hour_group['HOUR'], y=hour_group['Lat'], name=crime_name),
                row=1, col=i
            )
            i += 1
    
    fig.update_layout(
        title="Hourly/Weekly Crime Trends",
    )
    return fig
    

# Get Token for Mapbox & Read GeoJSON
token = 'pk.eyJ1IjoibmV3dXNlcmZvcmV2ZXIiLCJhIjoiY2o2M3d1dTZiMGZobzMzbnp2Z2NiN3lmdyJ9.cQFKe3F3ovbfxTsM9E0ZSQ'
app = dash.Dash(__name__)


app.layout = html.Div(children=[
    html.Div(
            children=[
                html.H2(children="Analisis de Crimen en Boston", className='h2-title'),
            ],
            className='study-browser-banner row'
    ),
    html.Div(
        className="row app-body",
        children=[
            # User Controls
            html.Div(
                className="twelve columns card",
                children=[
                    html.Div(
                        className="four columns card",
                        children=[
                            html.Div(
                                className="bg-white user-control",
                                children=[
                                    html.Div(
                                        className="padding-top-bot",
                                        children=[
                                            html.H6("Select Crime Type"),
                                            dcc.Dropdown(
                                                id="study-dropdown",
                                                multi=True,
                                                value=('Larceny',),
                                                options=[{'label': label.title(), 'value': label.title()} for label in df['OFFENSE_CODE_GROUP'].unique()]
                                            ),
                                            html.H6("Select a Date"),
                                            dcc.DatePickerRange(
                                                id="date-range",
                                                start_date=df['OCCURRED_ON_DATE'].min(),
                                                end_date=df['OCCURRED_ON_DATE'].max()
                                            ),
                                        ],
                                    ),
                                ],
                            )
                        ],
                    ),
                    html.Div(
                        className='eight columns card',
                        children=[
                            html.H1(children="Mapa geografico de crimen en Boston", style={'textAlign': 'center'}),
                            dcc.Graph(
                                id='map-plot',
                                figure={ 
                                    'data': [go.Scattermapbox()],
                                    'layout': go.Layout(
                                            mapbox_style="dark",
                                            mapbox_accesstoken=token,
                                            mapbox_zoom=10,
                                            margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                                            mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
                                        )
                                }
                            )
                        ]
                    )
                    ]
            ),
            html.Div(
                className='twelve columns card 2',
                children=[
                    html.Div(
                        className='six columns card',
                        children=[
                            dcc.Graph(
                            id='crime-total-graph',
                            figure={
                                'data': [go.Scatter()],
                            }
                        )],
                    ),
                    html.Div(
                        className='six columns card 2',
                        children=[
                            dcc.Graph(
                                id='crime-district-graph',
                                figure={
                                    'data': [go.Bar()],
                                }
                            )
                        ])
                        ]
                    )
                ]
            ),
            html.Div(
                className='twelve columns card 3',
                children=[
                    dcc.Graph(
                        id="crimes-weekly",
                        figure={
                            'data': [go.Scatter()]
                        }
                    )
                ]
            )
        ]
    )


@app.callback(
    dash.dependencies.Output('map-plot', 'figure'), # component with id map-plot will be changed, the 'figure' argument is updated
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_map(start_date, end_date, value):
    return { 
            'data': locations_by_crimetype(value, start_date, end_date),
            'layout': go.Layout(
                mapbox_style="dark",
                mapbox_accesstoken=token,
                mapbox_zoom=10,
                margin={'t': 0, 'l': 0, 'r': 0, 'b': 30},
                mapbox_center={"lat": df['Lat'][0], "lon": df['Long'][0]}
            )
        }



@app.callback(
    dash.dependencies.Output('crime-total-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_line_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_year(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Ocurrencia del Crimen en el tiempo'
            }
        }
    }


@app.callback(
    dash.dependencies.Output('crime-district-graph', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_bar_plot(start_date, end_date, value):
    return { 
        'data': crimes_by_district(value, start_date, end_date),
        'layout': {
            'title': {
                'text': 'Ocurrencia de Crimen por Distrito',

            }
        }
    }


@app.callback(
    dash.dependencies.Output('crimes-weekly', 'figure'),
    [
        dash.dependencies.Input('date-range', 'start_date'), # input with id date-picker-range and the start_date parameter
        dash.dependencies.Input('date-range', 'end_date'),
        dash.dependencies.Input('study-dropdown', 'value'),
    ]
)
def update_crimes_subplots(start_date, end_date, value):
    return crimes_week(value, start_date, end_date)

if __name__ == "__main__":
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2023 12:01:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:01:33] "GET /assets/styles.css?m=1572556444.0 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:01:33] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:01:33] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:01:33] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:01:33] "GET /_dash-component-suites/dash/dcc/async-datepicker.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:01:33] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:01:33] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2023 12:01:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:01:35] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2023 12:01:36] "PO

Esta función se complica un poco, pero sigue la misma estructura general que las anteriores. Para configurar el encuadre de las subtramas, usamos la función `make_subplots()` que toma el número de filas de la trama (solo 1) así como el número de columnas (7, una para cada día de la semana) .

Luego manipulamos y pasamos los datos necesarios para crear cada subtrama como un objeto `go.Scatter()`. Después de esto, todos los objetos `go.Scatter()` se agregan a nuestro diagrama principal usando la función `append_trace()`.

¡Continúe y agregue todo esto a `app.py` y ejecútelo para ver la versión final de nuestra aplicación!

## Conclusiones

In this case, we built an interactive dashboard which allows business users to gain insight into crime trends by crime type and by district across time. These insights can be used to mitigate crime in high-incident areas by facilitating increased preparedness by the police department.

Some interesting findings include:

1. Most larceny crimes are committed between 4 - 7 PM on weekdays. There is a sharp decline after 1 - 2 AM on most nights. This pattern is mostly consistent across crime types
2. A few exceptions to this include auto theft, which generally occurs after 8PM, and disorderly conduct, which happens mostly around midnight.

## Para recordar

En este caso, continuó aprovechando las habilidades que adquirió en el caso anterior sobre la configuración de los componentes de Dash y las funciones de devolución de llamada para completarlos dinámicamente. Además, aprendiste algunas cosas nuevas en Dash:

1. Usar una base de datos SQL como fuente de datos
2. Crear un diseño div jerárquico con múltiples divs anidados
3. Crear subtramas dentro de otra trama

En casos futuros, se conectará a Amazon Web Services (AWS) y se vinculará a una base de datos en la nube como fuente de datos.